# DNN Autoencoder example

The following tutorial demonstrates how to utilize safekit's multivariate DNN autoencoder to perform anomaly detection.

In [1]:
import tensorflow as tf
import numpy as np
import json
import sys

from safekit.batch import OnlineBatcher, split_batch
from safekit.graph_training_utils import ModelRunner, EarlyStop
from safekit.tf_ops import join_multivariate_inputs, dnn, multivariate_loss, eyed_mvn_loss
from safekit.util import make_feature_spec, make_loss_spec

tf.set_random_seed(408)
np.random.seed(408)

#整个代码都是在定义图（处理流程），而不是真的处理数据。

In [2]:
layer_list = [100, 50, 25, 50, 100]
lr = 5e-3
embed_ratio = 0.75
min_embed = 2
max_embed = 1000

Next, we load the JSON file describing the specifications for the data.

This JSON file describes a dictionary specifying the number of features in the input data; the categories corresponding to the features; whether the corresponding category is metadata, input, or output; and the indices which map these categories to specific features. This dictionary can later be used to ease interaction with the data when providing it as input to Tensorflow.

`datastart_index` specifies where the event counts begin in a single row of features; this is used by the minibatcher to ensure that it doesn't include metadata in the minibatches it produces.

In [5]:
dataspecs = json.load(open('../safekit/features/specs/agg/lanl_count_in_count_out_agg.json', 'r'))
datastart_index = dataspecs['counts']['index'][0]

Now that the data specifications have been loaded, we instantiate a batcher to divide the data into smaller portions. Since our dataset is rather large, we want to provide it to the model in small batches to avoid filling memory. Adjusting the minibatch size may also improve the model's performance. Here, we'll use a batch size of 256 data points.

In [6]:
#data = OnlineBatcher('/home/hutch_research/data/lanl/agg_feats/begin_no_weekends2.txt', 256, skipheader=True)
data = OnlineBatcher('/home/wxh/AnomalyDetectionModels/safekit-master/data_examples/lanl/agg_feats/lanl_agg_head.txt', 256, skipheader=True)

To put the data into a form that Tensorflow can efficiently process, we use `join_multivariate_inputs`; this function creates placeholders for the input data and defines operations in the Tensorflow graph that extract learned embeddings of categorical features and concatenate these embeddings together with the continuous features in the input. This defines the input that will be used by the DNN autoencoder.

In [7]:
feature_spec = make_feature_spec(dataspecs)
print(feature_spec)
x, ph_dict = join_multivariate_inputs(feature_spec, dataspecs, embed_ratio, min_embed, max_embed)
print(x, dataspecs)
print("ph_dict: ", ph_dict)
#ph_dict:placeholder_dict

{'continuous': [u'counts'], 'categorical': []}
(<tf.Tensor 'features:0' shape=(?, 134) dtype=float32>, {u'num_features': 137, u'counts': {u'index': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136], u'meta': 0, u'feature': 1, u'num_classes': 0, u'target': 1}, u'redteam': {u'index': [2], u'meta': 1, u'feature': 0, u'num_classes': 0, u'target': 0}, u'user': {u'index': [1], u'meta': 1, u'feature': 0, u'num_classes': 30000, u'target': 0}, u'time': {u'index': [0], u'meta': 1, u'featu

Now we instantiate the graph operations comprising the DNN autoencoder with a single call to the `dnn` function, which will return a Tensorflow variable mapping to the last layer of the DNN. This variable will then be used to define the model's loss. We specify the previously-defined input as the model's input, and provide a list which defines the number of hidden nodes at each hidden layer in the network.

In addition to model depth and width, this function has additional hyperparameters that can be tuned, including activation function, weight initialization range, and dropout factor.

In [8]:
#输出是dnn自编码器的最后一层，也就是解码后的数据，用于计算损失。
h = dnn(x, layer_list)
print('h: ', h)

('h: ', <tf.Tensor 'layer_4/h4:0' shape=(?, 100) dtype=float32>)


To determine how to compute losses, we use `make_loss_spec` to generate a specification dictionary mapping loss functions to features in the input. Then, `multivariate_loss` interprets this specification to define operations to compute feature-wise losses according to the data specifications—since the inputs can be a mixture of continuous and categorical features, their losses need to be defined accordingly. This function supports the use of three different loss functions—`eyed_mvn_loss`, `diag_mvn_loss`, and `full_mvn_loss`. Here we use `eyed_mvn_loss` to compute the squared error for predictions. The latter two use either diagonal or full covariance matrices to compute the Mahalonobis distance true values and predictions.

Once we define the graph operations that compute the squared error loss, we sum the losses over all features and average these losses over all data points in the minibatch. This is the scalar loss we will attempt to minimize using gradient descent.

In [9]:
loss_spec = make_loss_spec(dataspecs, eyed_mvn_loss)#确定每个维度怎么计算损失
loss_matrix = multivariate_loss(h, loss_spec, ph_dict)

loss_vector = tf.reduce_sum(loss_matrix, reduction_indices=1) #所有特征上误差相加
loss = tf.reduce_mean(loss_vector) #所有数据上误差取均值

To map losses back to our input features easily, we'll next define a function that we can call during the training loop that will write metadata and losses for each data point in the current minibatch. 

In [10]:
def write_results(data_dict, feat_loss, outfile):
    for d, u, t, l in zip(data_dict['time'].flatten().tolist(),
                          data_dict['user'].tolist(),
                          data_dict['redteam'].flatten().tolist(),
                          feat_loss.flatten().tolist()):
        outfile.write('%s %s %s %s\n' % (int(d), u, int(t), l))

Now we instantiate a `ModelRunner` object, which provides a simple interface for interacting with the Tensorflow session. Instantiating this object will define the optimizer Tensorflow will use for gradient descent and initialize all of the variables in the Tensorflow graph. We can then use the `train_step` method on this object to perform an optimization step or the `eval` method to retrieve the values of arbitrary tensors in the graph.

The `loss_feats` variable specifies the names of the features over which we are computing losses. In this case, it is a single category—the counts of the categorical features—though in instances with a mixture of categorical and continuous features, more than one category could be represented here.

In order to record the losses for all of the features, we define a list `eval_tensors` that contains tensors whose values we want to retrieve during training. We'll provide this list to the `ModelRunner`'s `eval` method during the training loop to compute these tensors, then record their values with the `write_results` function defined previously.

In [12]:
# other args incl. learning rate, optimizer, decay rate...
model = ModelRunner(loss, ph_dict, learnrate=lr, opt='adam')

loss_feats = [triple[0] for triple in loss_spec]

# list of tensors we want to retrieve at each training step; can also add loss_matrix to this
eval_tensors = [loss, loss_vector]

Now we begin training our model. We start by defining a stopping criteria for training using the `EarlyStop` object; if our model's performance doesn't improve after 20 training steps, the `check_error` function we instantiate will return `False`, and training will be discontinued.

Inside the training loop, `split_batch` is first used to construct a dictionary for Tensorflow that maps features to the placeholders used to feed data into the computational graph during training. Since our targets are defined separately from the inputs provided to our batcher, we add these to the dictionary.

We retrieve the losses for the current batch, then perform a training step to perform gradient descent over a single batch of inputs. This process repeats until either the batcher has reached the end of the input file, the stopping criteria has been met, or the model's error has diverged to infinity.

In [16]:
check_error = EarlyStop(20)

cur_loss = sys.float_info.max # starting with largest loss possible
raw_batch = data.next_batch()
training = check_error(raw_batch, cur_loss)

outfile = open('results', 'w')

while training:
    data_dict = split_batch(raw_batch, dataspecs)
    targets = {'target_' + name : data_dict[name] for name in loss_feats}
    data_dict.update(targets)
    cur_loss, feat_loss = model.eval(data_dict, eval_tensors)
    model.train_step(data_dict)
    
    write_results(data_dict, feat_loss, outfile)
    print('index: %s, loss: %.4f' % (data.index, cur_loss))
    raw_batch = data.next_batch()
    training = check_error(raw_batch, cur_loss)
    
outfile.close()

Done Training. End of data stream.